In [1]:
! pip install openai-agents
! pip install resend

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import resend
import os
from agents import Tool, Agent
from typing import Dict
from IPython.display import display, Markdown

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ["OPENAI_API_KEY"]= user_secrets.get_secret("OPENAI_API_KEY")
os.environ["RESEND_API_KEY"] = user_secrets.get_secret("RESEND_API_KEY")

resend.api_key = os.environ["RESEND_API_KEY"]

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Comparison of the best AI Agent development tools in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of November 2025, several AI agent development tools have emerged, each offering unique features tailored to different user needs.

**AutoGPT** is an open-source autonomous AI agent that utilizes OpenAI's large language models, such as GPT-4, to achieve user-specified goals. It autonomously breaks down tasks and employs tools like web browsing and file management. However, it faces challenges like tendency to get stuck in loops, hallucinate information, and incur high operational costs due to reliance on paid APIs. ([en.wikipedia.org](https://en.wikipedia.org/wiki/AutoGPT?utm_source=openai))

**Kruti** is a multilingual AI agent developed by Ola Krutrim, designed to perform real-world tasks such as booking taxis and ordering food by integrating directly with various online services. It is notable for its ability to understand and respond in multiple Indian languages. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

**AWS Kiro** is an AI-powered integrated development environment (IDE) introduced by Amazon Web Services. It employs intelligent agents to break down project prompts into structured components, enabling more effective implementation, testing, and change tracking. Key features include automatic project planning, support for Model Context Protocol (MCP), steering rules for AI behavior, and ad-hoc agentic chat for various tasks. ([techradar.com](https://www.techradar.com/pro/aws-launches-kiro-an-agentic-ai-ide-to-end-the-chaos-of-vibe-coding?utm_source=openai))

**ToolBrain** is a flexible reinforcement learning framework for agentic tools, designed to ease the training of agents in utilizing tools through flexible reinforcement learning. It supports a wide range of training strategies, including RL algorithms such as GRPO and DPO, as well as supervised learning. ToolBrain enables custom reward callables directly on an agent's execution traces or utilizes an automated LLM-as-a-judge system for reward generation. ([arxiv.org](https://arxiv.org/abs/2510.00023?utm_source=openai))

**AlphaEvolve** is an evolutionary coding agent developed by Google DeepMind, aiming to autonomously discover and refine algorithms through a combination of large language models and evolutionary computation. It is designed as a general-purpose system, unlike domain-specific predecessors like AlphaFold or AlphaTensor. ([en.wikipedia.org](https://en.wikipedia.org/wiki/AlphaEvolve?utm_source=openai))

**Manus** is an autonomous AI agent developed by Butterfly Effect Technology, designed to independently execute complex real-world tasks without continuous human supervision. It is described as one of the first fully autonomous AI agents capable of independent reasoning, dynamic planning, and autonomous decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

**Simpliflow** is a lightweight, open-source Python framework designed for the rapid development and orchestration of linear, deterministic agentic workflows through a declarative, JSON-based configuration. Its modular architecture decouples agent management, workflow execution, and post-processing, promoting ease of use and extensibility. ([arxiv.org](https://arxiv.org/abs/2510.10675?utm_source=openai))

**Jules** is an AI-powered coding agent developed by Google, built on the Gemini 2.5 Pro model. It is designed not only for professional developers but also for casual users interested in automation, app creation, or web design—even without programming experience. Jules is accessible via Google Labs with a free introductory tier for hobby use, and Pro and Ultra tiers offering significantly higher capacities for professionals and enterprise-scale needs. ([tomsguide.com](https://www.tomsguide.com/ai/googles-powerful-ai-coding-agent-jules-is-now-available-to-everyone-heres-why-its-not-just-for-developers?utm_source=openai))

**OpenAI Codex** is a developer agent that has been integrated across multiple development environments, including GitHub, terminals, web interfaces, and IDEs like VS Code and Cursor. This integration allows developers to engage with Codex directly in their existing workflow environments using their ChatGPT accounts, ensuring seamless state management between local and cloud-based activities. ([itpro.com](https://www.itpro.com/business/business-strategy/openais-codex-developer-agent-just-got-a-big-update?utm_source=openai))

**Google's Gemini API** has introduced improved support for structured outputs, notably through the integration of JSON Schema. This update simplifies the development of multi-agent AI systems by ensuring AI-generated responses conform to consistent schemas, which is crucial for tasks like data extraction and seamless agent-to-agent communication. ([androidcentral.com](https://www.androidcentral.com/apps-software/ai/google-is-making-it-easier-to-use-the-gemini-api-in-multi-agent-workflows?utm_source=openai))

**$Agent^2$** is a novel agent-generates-agent framework that achieves fully automated reinforcement learning agent design through intelligent LLM-driven generation. The system autonomously transforms natural language task descriptions and environment code into comprehensive, high-performance reinforcement learning solutions without human intervention. ([arxiv.org](https://arxiv.org/abs/2509.13368?utm_source=openai))

**AutoAgent** is a fully-automated and highly self-developing framework that enables users to create and deploy LLM agents through natural language alone. Operating as an autonomous Agent Operating System, AutoAgent comprises four key components: Agentic System Utilities, LLM-powered Actionable Engine, Self-Managing File System, and Self-Play Agent Customization module. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These tools represent the forefront of AI agent development, each contributing unique capabilities to the evolving landscape of autonomous AI systems. 

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
message = "Comparison of the best AI Agent development tools in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find a list of AI agent development tools that are expected to be popular or highly rated in 2025.', query='best AI agent development tools 2025'), WebSearchItem(reason='To identify features, advantages, and user feedback on top AI agent development platforms in 2025.', query='comparison of AI agent development tools features 2025'), WebSearchItem(reason='To gather insights from industry experts and reports on the landscape of AI development tools in 2025.', query='AI agent development tools industry report 2025')]


In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> dict:
    """
    Send an email with the given subject and HTML body using Resend.
    """
    response = client.emails.send(from_email="onboarding@resend.dev",to=["evil32894@gmail.com"],subject=subject,html=html_body)
    return {"status": "success", "id": response.id}



INSTRUCTIONS = """You are an email assistant. Your task is to send a single, well-formatted HTML email
based on a provided report. Convert the report into clean, professional HTML, choose an appropriate
subject line, and use your email-sending tool to deliver it. Focus on clarity, readability, and
presentation; do not include extra commentary."""


email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


In [8]:
INSTRUCTIONS = (
    "You are a senior researcher responsible for producing a detailed, cohesive report based on a research query. "
    "You will receive the original query along with initial research from a research assistant.\n\n"
    "Steps:\n"
    "1. Create a structured outline that defines the flow and sections of the report.\n"
    "2. Write the full report following the outline. Ensure it is detailed, thorough, and insightful.\n"
    "3. Return the final output in markdown format. The report should be at least 1000 words, "
    "equivalent to 5-10 pages of content.\n\n"
    "Focus on clarity, logical flow, and completeness. Avoid unnecessary commentary; the content should be professional and ready for presentation."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")




writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [9]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [10]:
query ="Cutting-edge AI agent frameworks and platforms 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")



Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
